In [ ]:
from google.colab import drive
drive.mount('/content/drive')


KeyboardInterrupt: 

In [ ]:
!pip install opencv-python-headless

In [ ]:
import os
import cv2
import numpy as np

In [ ]:
import os
import cv2
import numpy as np

def extract_frames(video_path, frame_count=10):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = total_frames // frame_count

    for i in range(frame_count):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * interval)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (64, 64))
            frames.append(frame)
        else:
            break
    cap.release()
    return np.array(frames)

def load_data(data_dir):
    X = []
    y = []
    labels = {'Defensive Shot': 0, 'Pull Shot': 1, 'Drive Shot': 2, 'Flick Shot': 3}

    for label in labels:
        videos_dir = os.path.join(data_dir, label)
        videos = os.listdir(videos_dir)
        for video in videos:
            video_path = os.path.join(videos_dir, video)
            frames = extract_frames(video_path)
            if len(frames) == 10:
                X.append(frames)
                y.append(labels[label])

    X = np.array(X)
    y = np.array(y)
    return X, y

In [ ]:
data_dir = '/content/drive/My Drive/ShotDetection'
X, y = load_data(data_dir)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train / 255.0
X_val = X_val / 255.0

# (batch_size, time_steps, height, width, channels)
X_train = np.reshape(X_train, (X_train.shape[0], 10, 64, 64, 3))
X_val = np.reshape(X_val, (X_val.shape[0], 10, 64, 64, 3))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, ConvLSTM2D, MaxPooling3D, TimeDistributed, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

def create_convlstm_model(input_shape):
    model = Sequential()

    model.add(ConvLSTM2D(filters=4, kernel_size=(3, 3), activation='tanh', return_sequences=True, input_shape=input_shape))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same'))

    model.add(TimeDistributed(Dropout(0.2)))
    model.add(ConvLSTM2D(filters=8, kernel_size=(3, 3), activation='tanh', return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same'))

    model.add(TimeDistributed(Dropout(0.2)))
    model.add(ConvLSTM2D(filters=14, kernel_size=(3, 3), activation='tanh', return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same'))

    model.add(TimeDistributed(Dropout(0.2)))
    model.add(ConvLSTM2D(filters=16, kernel_size=(3, 3), activation='tanh', return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same'))

    model.add(Flatten())
    model.add(Dense(4, activation='softmax'))  # 4 classes: Defensive Shot, Pull Shot, Drive Shot and Flick Shot

    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

    return model

input_shape = X_train.shape[1:]
model = create_convlstm_model(input_shape)

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 10, 62, 62, 4)     1024      
                                                                 
 max_pooling3d (MaxPooling3  (None, 10, 31, 31, 4)     0         
 D)                                                              
                                                                 
 time_distributed (TimeDist  (None, 10, 31, 31, 4)     0         
 ributed)                                                        
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 10, 29, 29, 8)     3488      
                                                                 
 max_pooling3d_1 (MaxPoolin  (None, 10, 15, 15, 8)     0         
 g3D)                                                            
                                                        

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_val, y_val), callbacks=[early_stopping])

Epoch 1/10
64/64 [==============================] - 200s 3s/step - loss: 1.3789 - accuracy: 0.2613 - val_loss: 1.3795 - val_accuracy: 0.2656
Epoch 2/10
64/64 [==============================] - 192s 3s/step - loss: 1.3794 - accuracy: 0.2838 - val_loss: 1.3761 - val_accuracy: 0.2695
Epoch 3/10
64/64 [==============================] - 195s 3s/step - loss: 1.3742 - accuracy: 0.2994 - val_loss: 1.3785 - val_accuracy: 0.3047
Epoch 4/10
64/64 [==============================] - 193s 3s/step - loss: 1.3460 - accuracy: 0.3493 - val_loss: 1.3134 - val_accuracy: 0.3555
Epoch 5/10
64/64 [==============================] - 192s 3s/step - loss: 1.3096 - accuracy: 0.3669 - val_loss: 1.2984 - val_accuracy: 0.3789
Epoch 6/10
64/64 [==============================] - 195s 3s/step - loss: 1.2774 - accuracy: 0.3992 - val_loss: 1.2692 - val_accuracy: 0.3828
Epoch 7/10
64/64 [==============================] - 194s 3s/step - loss: 1.2749 - accuracy: 0.3836 - val_loss: 1.2669 - val_accuracy: 0.3633
Epoch 8/10
64

In [ ]:
model.save('/content/drive/My Drive/ShotDetection/convlstm_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import tensorflow as tf

model_path = '/content/drive/My Drive/ShotDetection/convlstm_model.h5'
model = tf.keras.models.load_model(model_path)

In [ ]:
from google.colab import files

uploaded = files.upload()

# Move the uploaded video to a specific directory
os.makedirs('videos', exist_ok=True)
for file_name in uploaded.keys():
    os.rename(file_name, os.path.join('videos', file_name))

Saving 320.mp4 to 320.mp4


In [ ]:
video_path = 'videos/320.mp4'

new_frames = extract_frames(video_path)
new_frames = np.expand_dims(new_frames, axis=0)
new_frames = np.expand_dims(new_frames, axis=-1)

prediction = model.predict(new_frames)
predicted_class = np.argmax(prediction, axis=1)

print(prediction)
shot = {0: 'Defensive Shot', 1: 'Pull Shot', 2: 'Drive Shot', 3: 'Flick Shot'}
print(shot[predicted_class[0]])

1/1 [==============================] - 0s 133ms/step
[[0.12038562 0.090731   0.38469142 0.404192  ]]
Flick Shot
